In [ ]:
!pip install geocoder

In [ ]:
import geopy.geocoders
from geopy.geocoders import Nominatim 

def address_to_coordinates(address):
    geolocator = geopy.geocoders.Nominatim(user_agent='geoapiExercises')
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

coordinates = address_to_coordinates('台北市信義區逸仙路50巷22號')
print(coordinates)  # (37.4224764, -122.0842499)


In [ ]:
import geocoder
g = geocoder.google('台北市信義區逸仙路50巷22號')
print(g.latlng)

In [ ]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Mountain View, CA'}
r = requests.get(url, params=params)
results = r.json()['results']
location = results[0]['geometry']['location']
location['lat'], location['lng']

In [ ]:
import datetime
# 取得當前日期
today = datetime.datetime.today().date()
print(today)

# 計算隔天日期
tomorrow = today + datetime.timedelta(days=1)

# 輸出隔天日期
print(tomorrow)

In [ ]:
!pip3 install openpyxl

In [ ]:
import pandas as pd
import openpyxl
def population(loc):
    df = pd.read_excel('C:/Users/student/Desktop/my_project/111年09月 ( 鄉鎮市區 )人口密度.xlsx')
    x = df.loc[df['地區']==f'{loc}',['人口密度'][0]]
    return x

In [ ]:
population('新北市永和區')

In [ ]:
import numpy as np
import pandas as pd 
from scipy import spatial
import pandas as pd
import openpyxl
import datetime

stream = pd.read_csv('110.csv', low_memory=False)
stream['總計_流量(PCU)'] = stream['總計_流量(PCU)'].str.findall('\d').str.join('').astype(int)
stream['ID']=stream['年份'].astype(str)+stream['調查站_座標_經度'].astype(str)+stream['調查站_座標_緯度'].astype(str)
stream = stream.groupby('ID').mean()
KDTree_stream = spatial.KDTree(stream[['調查站_座標_經度','調查站_座標_緯度']])

def population(loc):
    df = pd.read_excel('C:/Users/student/Desktop/my_project/111年09月 ( 鄉鎮市區 )人口密度.xlsx')
    density = df.loc[df['地區']==f'{loc}',['人口密度'][0]]
    # 取得當前日期
    # today = datetime.datetime.today().date()
    # print(today)

    # 計算隔天日期
    tomorrow = today + datetime.timedelta(days=1)

    # 輸出隔天日期
    # print(tomorrow)

    dic = {'year' : f'{tomorrow.year}', 
        'month' : f'{tomorrow.month}',
        'day' : f'{tomorrow.day}',
        'hour' : '0',
        'long' : '120',
        'lat' : '25',
        'pop' : f'{density}'}
    return density

def dataFill(dic):
    
    year = dic['year']
    month = dic['month']
    day = dic['day']
    hour = dic['hour']
    long = dic['long']
    lat = dic['lat']
    pop = dic['pop']
    
    
    input_columns = ['發生年度','發生月份', '發生日期', '發生時間', '經度', '緯度',  'population_density']
    columns = ['dow', 'doy', '元旦', '春節', '二二八', '兒童清明', '端午', '中秋', '雙十', '發生年度','發生月份', '發生日期', '發生時間', '經度', '緯度', '氣溫(℃)_x', '測站氣壓(hPa)_x','相對溼度(%)_x', '降水量(mm)_x', '風向(360degree)_x', '風速(m/s)_x', 'population_density', '流量加權']

    input_data = pd.DataFrame(columns=input_columns, index=[1]); input_data.iloc[0,:] = [year, month, day, hour, long, lat,pop]; input_data.astype(str)
    df = pd.DataFrame(columns = columns, index=[i for i in range(24)])

    df.經度 = long
    df.緯度 = lat
    df.population_density = pop

    datetime = input_data.apply((lambda x: x.發生年度+'-'+ x.發生月份+'-'+x.發生日期+' '+x.發生時間+':00:00'), axis = 1)
    time = pd.datetime.strptime(datetime[1], '%Y-%m-%d %H:%M:%S')
    time_range = pd.date_range(start=time, periods=24, freq='H')
    time_strings = time_range.strftime('%Y-%m-%d %H:%M:%S')

    df['dow'] = time_range.dayofweek
    df['doy'] = time_range.dayofyear
    df['發生年度'] = time_range.year
    df['發生月份'] = time_range.month
    df['發生日期'] = time_range.day
    df['發生時間'] = time_range.hour

    def taiwan_festival(x: int) -> pd.Series:

        sr = pd.Series([0,0,0,0,0,0,0],index=['元旦','春節','二二八','兒童清明','端午','中秋','雙十'])

        def _2018( x, sr):
            if x == 20180101:
                sr['元旦'] = 1
            elif 20180215 <= x <= 20180220:
                sr['春節'] = 1
            elif x == 20180228:
                sr['二二八'] = 1
            elif 20180404 <= x <= 20180408:
                sr['兒童清明'] = 1
            elif 20180616 <= x <= 20180618:
                sr['端午'] = 1
            elif 20180922 <= x <= 20180924:
                sr['中秋'] = 1
            elif x == 20181010:
                sr['雙十'] = 1
            elif 20181229 <= x <= 20181231:
                sr['元旦'] = 1
            return sr

        def _2019( x, sr):
            if x == 20190101:
                sr['元旦'] = 1
            elif 20190202 <= x <= 20190210:
                sr['春節'] = 1
            elif 20190228 <= x <= 20190303:
                sr['二二八'] = 1
            elif 20190404 <= x <= 20190407:
                sr['兒童清明'] = 1
            elif 20190607 <= x <= 20190609:
                sr['端午'] = 1
            elif 20190913 <= x <= 20190915:
                sr['中秋'] = 1
            elif 20191010 <= x <= 20191013:
                sr['雙十'] = 1
            return sr


        def _2020( x, sr):
            if x == 20200101:
                sr['元旦'] = 1
            elif 20200123 <= x <= 20200129:
                sr['春節'] = 1
            elif 20200228 <= x <= 20200301:
                sr['二二八'] = 1
            elif 20200402 <= x <= 20200405:
                sr['兒童清明'] = 1
            elif 20200625 <= x <= 20200628:
                sr['端午'] = 1
            elif 20201001 <= x <= 20201004:
                sr['中秋'] = 1
            elif 20201009 <= x <= 20201011:
                sr['雙十'] = 1
            return sr

        def _2021( x, sr):
            if 20210101 <= x <= 20210103:
                sr['元旦'] = 1
            elif 20210210  <= x <= 20210216:
                sr['春節'] = 1
            elif 20210227 <= x <= 20210301:
                sr['二二八'] = 1
            elif 20210402 <= x <= 20210405:
                sr['兒童清明'] = 1
            elif 20210612 <= x <= 20210614:
                sr['端午'] = 1
            elif 20210918 <= x <= 20210921:
                sr['中秋'] = 1
            elif 20211009 <= x <= 20211011:
                sr['雙十'] = 1
            elif x == 20211231:
                sr['元旦'] = 1
            return sr

        def _2022( x, sr):
            if 20220101 <= x <= 20220102:
                sr['元旦'] = 1
            elif 20220129 <= x <= 20220206:
                sr['春節'] = 1
            elif 20220226 <= x <= 20220228:
                sr['二二八'] = 1
            elif 20220402 <= x <= 20220405:
                sr['兒童清明'] = 1
            elif 20220603 <= x <= 20220605:
                sr['端午'] = 1
            elif 20220909 <= x <= 20220911:
                sr['中秋'] = 1
            elif 20221008 <= x <= 20221010:
                sr['雙十'] = 1
            elif x == 20221231:
                sr['元旦'] = 1
            return sr

        def _2023( x, sr):
            if 20230101 <= x <= 20230102:
                sr['元旦'] = 1
            elif 20230120 <= x <= 20230129:
                sr['春節'] = 1
            elif 20230225 <= x <= 20230228:
                sr['二二八'] = 1
            elif 20230401 <= x <= 20230405:
                sr['兒童清明'] = 1
            elif 20230622 <= x <= 20230625:
                sr['端午'] = 1
            elif 20230929 <= x <= 20231001:
                sr['中秋'] = 1
            elif 20231007 <= x <= 20231010:
                sr['雙十'] = 1
            elif 20231230 <= x <= 20231231:
                sr['元旦'] = 1
            return sr

        year = x // 10000

        if year == 2018:
            sr = _2018(x, sr)
        elif year == 2019:
            sr = _2019(x, sr)
        elif year == 2020:
            sr = _2020(x, sr)
        elif year == 2021:
            sr = _2021(x, sr)
        elif year == 2022:
            sr = _2022(x, sr)
        elif year == 2023:
            sr = _2023(x, sr)

        return sr

    date_int = df.發生年度.astype(int)*10000 + df.發生月份.astype(int)*100 + df.發生日期.astype(int)
    df[['元旦', '春節', '二二八', '兒童清明', '端午', '中秋', '雙十']] = date_int.apply((taiwan_festival))

    def nearest(x: pd.DataFrame):
        distance = KDTree_stream.query((x.經度, x.緯度),k=3)[0]
        index = KDTree_stream.query((x.經度, x.緯度),k=3)[1]
        df = stream.iloc[index,:]
        df['與事故點距離'] = pd.Series(distance,index=df.index)
        df = df[['調查站_座標_經度','調查站_座標_緯度','與事故點距離','總計_流量(PCU)']]
        df = pd.concat([df.iloc[0,],df.iloc[1,],df.iloc[2,]])
        return df

    stream_data = df.apply(nearest, axis=1)
    stream_data.columns = ['調查站1_座標_經度','調查站1_座標_緯度','調查站1與事故點距離','調查站1總計_流量(PCU)','調查站2_座標_經度','調查站2_座標_緯度','調查站2與事故點距離','調查站2總計_流量(PCU)','調查站3_座標_經度','調查站3_座標_緯度','調查站3與事故點距離','調查站3總計_流量(PCU)']

    stream_data['流量加權'] = stream_data['調查站1總計_流量(PCU)']*(1/(stream_data.調查站1與事故點距離))/(1/(stream_data.調查站1與事故點距離)+1/(stream_data.調查站2與事故點距離)+1/(stream_data.調查站3與事故點距離)) +\
                stream_data['調查站2總計_流量(PCU)']*(1/(stream_data.調查站2與事故點距離))/(1/(stream_data.調查站1與事故點距離)+1/(stream_data.調查站2與事故點距離)+1/(stream_data.調查站3與事故點距離)) +\
                stream_data['調查站3總計_流量(PCU)']*(1/(stream_data.調查站3與事故點距離))/(1/(stream_data.調查站1與事故點距離)+1/(stream_data.調查站2與事故點距離)+1/(stream_data.調查站3與事故點距離))

    df.流量加權 = stream_data.流量加權
    
    return df

In [ ]:
dataFill(dic)

In [ ]:
# https://www.visualcrossing.com
def weatherAPI(df1):
    # 密鑰
#     key = '4KD3GK5WQ8HX23YXVEVS3ZYPM'
    key = 'NZKHHNNZV9ZXDRGDM4NJTT9YR'
    # 緯度，經度
    a1 = f"{df1.iloc[0,14]},{df1.iloc[0,13]}"
    # 時間
    t1 = f"{df1.iloc[0,9]}-{df1.iloc[0,10]}-{df1.iloc[0,11]}"
    t2 = f"{df1.iloc[0,9]}-{df1.iloc[0,10]}-{df1.iloc[0,11]}"
    # 單位
    unitGroup = 'metric'
    # 語言
    lang = 'zh'
    # 需求資料
    include = 'hours'
    # 元素
    elements = 'datetime,pressure,humidity,temp,winddir,windspeed,precip'

    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{a1}/{t1}/{t2}?key={key}&unitGroup={unitGroup}&lang={lang}&include={include}&elements={elements}"


    response = requests.get(url)
#     print(response.status_code)
    if response.status_code == 200:
        data = json.loads(response.text)
#         pprint(data)
        a = pd.DataFrame(data['days'][0]['hours'])
        df1['氣溫(℃)_x'] = a['temp']
        df1['測站氣壓(hPa)_x'] = a['pressure']
        df1['相對溼度(%)_x'] = a['humidity']
        df1['降水量(mm)_x'] = a['precip']
        df1['風向(360degree)_x'] = a['winddir']
        df1['風速(m/s)_x'] = a['windspeed']*1000/3600
        df1 = df1.round({'風速(m/s)_x':1})
#         pprint(f"查詢日期 = {data['days'][0]['datetime']}")
#         pprint(f"濕度(%) = {data['days'][0]['humidity']}")
#         pprint(f"大氣壓力(hPa) = {data['days'][0]['pressure']}")
#         pprint(f"溫度(°C) = {data['days'][0]['temp']}")
#         pprint(f"風向(°) = {data['days'][0]['winddir']}")
#         pprint(f"風速(km/h) = {data['days'][0]['windspeed']}")
#         print('+'*10)

#         for i in range(0,24):
#             df1.loc[df1.發生時間==i,'氣溫(℃)_x']=data['days'][0]['hours'][i]['temp']
#             df1.loc[df1.發生時間==i,'測站氣壓(hPa)_x']=data['days'][0]['hours'][i]['pressure']
#             df1.loc[df1.發生時間==i,'相對溼度(%)_x']=data['days'][0]['hours'][i]['humidity']
#             df1.loc[df1.發生時間==i,'風向(360degree)_x']=data['days'][0]['hours'][i]['winddir']
#             df1.loc[df1.發生時間==i,'風速(m/s)_x']=data['days'][0]['hours'][i]['windspeed']
            
#             pprint(f"查詢小時 = {data['days'][0]['hours'][i]['datetime']}")
#             pprint(f"濕度(%) = {data['days'][0]['hours'][i]['humidity']}")
#             pprint(f"大氣壓力(hPa) = {data['days'][0]['hours'][i]['pressure']}")
#             pprint(f"溫度(°C) = {data['days'][0]['hours'][i]['temp']}")
#             pprint(f"風向(°) = {data['days'][0]['hours'][i]['winddir']}")
#             pprint(f"風速(km/h) = {data['days'][0]['hours'][i]['windspeed']}")
#             pprint(f"降雨量(mm) = {data['days'][0]['hours'][i]['precip']}")
#             print('-'*10)
        return df1

In [2]:
import geocoder
x = geocoder.arcgis('新北市新莊區思源路179號').latlng

x

[25.045297307251495, 121.45972677974738]

In [21]:
import pandas as pd
df = pd.read_excel('11109density.xlsx')
df
float(df[df['地區']== '新北市新莊區' ]['人口密度'])

21219.81

In [1]:
import use_model.xgdata as xg
import geocoder

coo = geocoder.arcgis('桃園市中壢區遠東路135號').latlng
o = xg.population('桃園市中壢區',coo)
o

c:\Users\student\Desktop\my_project\forgit\BDSE28-website\use_model\xgdata.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stream = stream.groupby('ID').mean()


{'year': '2023',
 'month': '1',
 'day': '5',
 'hour': '13',
 'long': '121.26294001995723',
 'lat': '24.969200014588054',
 'pop': '5506.72'}

In [2]:
dicdata= {
  "name": "John Smith",'sam'
  "age": 30,
  "city": "New York"
}

dicdata.name

AttributeError: 'dict' object has no attribute 'name'

In [ ]:
import pandas as pd
import openpyxl
df = pd.read_excel('111年09月 ( 鄉鎮市區 )人口密度.xlsx')
density = df.loc[df['地區']=='新北市永和區',['人口密度'][0]][0]

density

In [ ]:
import xgboost as xgb
def pred_loc(df):
    model = xgb.Booster()
    model.load_model('model_best_xgb(無做特徵縮減).pkl')
    y_pred = model.predict(df)
    return y_pred

In [19]:
data={'key1':1}
for i in [1,2,3,4,5]:
    data['key1']= data['key1']+i
data

{'key1': 16}

In [3]:
import geocoder
import use_model.xgdata as xg
import xgboost as xgb
import pickle
import pandas as pd
addr = '嘉義縣阿里山鄉中山村6鄰排雲101號'
coo = geocoder.arcgis(addr).latlng
xgdic = xg.population('嘉義縣阿里山鄉',coo)
df1 = xg.dataFill(xgdic)
forxg = xg.weatherAPI(df1)

def pred_loc(forxg):
    with open('model_best_xgb(無做特徵縮減).pkl','rb') as f:
        clf_load = pickle.load(f)
        prob = pd.DataFrame(clf_load.predict_proba(forxg)[:,1])
        
        def rank(x):
            
            if 0<=x < 0.1941:
                x = 'rank1'
            elif 0.1941<=x<0.2141:
                x = 'rank2'
            elif 0.2141 <= x <0.2546:
                x = 'rank3'
            elif 0.2546<= x<0.3274:
                x = 'rank4'
            elif 0.3274<= x <0.4806:
                x = 'rank5'
            elif 0.4806<=x <0.6800:
                x = 'rank6'
            elif 0.6800<=x <0.7526:
                x = 'rank7'
            elif 0.7526 <=x <0.8030:
                x = 'rank8'
            elif 0.8030<=x < 0.8130:
                x = 'rank9'
            elif 0.8130 <=x < 1:
                x = 'rank10'

            return x
        level = prob.iloc[:,0].astype(float).apply(rank)
        hour = forxg.iloc[:,12]
        # weather = forxg.iloc[:,-8:-2]
        # df = pd.concat([hour, weather, level],axis=1)
        df = pd.concat([hour,level],axis=1)
        df = df.rename(columns={0: 'rank'})
        return df

acc_rank = pred_loc(forxg)


c:\Users\student\Desktop\my_project\forgit\BDSE28-website\use_model\xgdata.py:60: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  time = pd.datetime.strptime(datetime[1], '%Y-%m-%d %H:%M:%S')
C:\Users\student\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\student\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn

In [4]:
acc_rank

# 使用.values属性获取DataFrame中所有值的NumPy数组
values = acc_rank.values

# 将NumPy数组转换为Python列表
result = values.tolist()


In [48]:
acc_rank= acc_rank.rename(columns={0: 'rank'})
acc_rank

,發生時間,rank
0,15,rank7
1,16,rank7
2,17,rank7
3,18,rank6
4,19,rank2
5,20,rank2
6,21,rank2
7,22,rank2
8,23,rank2
9,0,rank1


In [54]:
a = {'key1':1}
a.update(acc_rank.to_dict(orient='list' ))
a

{'key1': 1,
 '發生時間': [15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14],
 'rank': ['rank7',
  'rank7',
  'rank7',
  'rank6',
  'rank2',
  'rank2',
  'rank2',
  'rank2',
  'rank2',
  'rank1',
  'rank1',
  'rank1',
  'rank1',
  'rank1',
  'rank1',
  'rank5',
  'rank7',
  'rank7',
  'rank7',
  'rank7',
  'rank7',
  'rank7',
  'rank7',
  'rank7']}

In [29]:
time=[]
mydict={}
for i in result:
    time.append(i[0])
mydict['time']=time

print(mydict)

{'time': [15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}


In [24]:
mydict= {}
for i in result:
    mydict = mydict['time'].append(i[0])

print(mydict)

AttributeError: 'str' object has no attribute 'append'

In [4]:
import numpy as np
import pandas as pd 
from scipy import spatial
import pandas as pd
import openpyxl
import datetime
import requests , json

stream = pd.read_csv('110.csv', low_memory=False)
stream['總計_流量(PCU)'] = stream['總計_流量(PCU)'].str.findall('\d').str.join('').astype(int)
stream['ID']=stream['年份'].astype(str)+stream['調查站_座標_經度'].astype(str)+stream['調查站_座標_緯度'].astype(str)
stream = stream.groupby('ID').mean()
KDTree_stream = spatial.KDTree(stream[['調查站_座標_經度','調查站_座標_緯度']])

def population(county_dist , coo):
    df_den = pd.read_excel('11109density.xlsx')
    density = float(df_den[df_den['地區']== county_dist ]['人口密度'])
    # 取得當前日期
    today = datetime.datetime.today().date()
    
    # 計算隔天日期
    tomorrow = today + datetime.timedelta(days=1)

    dic = {'year' : f'{tomorrow.year}', 
        'month' : f'{tomorrow.month}',
        'day' : f'{tomorrow.day}',
        'hour' : '0',
        'long' : f'{coo[1]}',
        'lat' : f'{coo[0]}',
        'pop' : f'{density}'}
    return dic

def dataFill(dic):
    
    year = dic['year']
    month = dic['month']
    day = dic['day']
    hour = dic['hour']
    long = dic['long']
    lat = dic['lat']
    pop = dic['pop']
    
    
    input_columns = ['發生年度','發生月份', '發生日期', '發生時間', '經度', '緯度',  'population_density']
    columns = ['dow', 'doy', '元旦', '春節', '二二八', '兒童清明', '端午', '中秋', '雙十', '發生年度','發生月份', '發生日期', '發生時間', '經度', '緯度', '氣溫(℃)_x', '測站氣壓(hPa)_x','相對溼度(%)_x', '降水量(mm)_x', '風向(360degree)_x', '風速(m/s)_x', 'population_density', '流量加權']

    input_data = pd.DataFrame(columns=input_columns, index=[1]); input_data.iloc[0,:] = [year, month, day, hour, long, lat,pop]; input_data.astype(str)
    df = pd.DataFrame(columns = columns, index=[i for i in range(24)])

    df.經度 = long
    df.緯度 = lat
    df.population_density = pop

    datetime = input_data.apply((lambda x: x.發生年度+'-'+ x.發生月份+'-'+x.發生日期+' '+x.發生時間+':00:00'), axis = 1)
    time = pd.datetime.strptime(datetime[1], '%Y-%m-%d %H:%M:%S')
    time_range = pd.date_range(start=time, periods=24, freq='H')
    time_strings = time_range.strftime('%Y-%m-%d %H:%M:%S')

    df['dow'] = time_range.dayofweek
    df['doy'] = time_range.dayofyear
    df['發生年度'] = time_range.year
    df['發生月份'] = time_range.month
    df['發生日期'] = time_range.day
    df['發生時間'] = time_range.hour

    def taiwan_festival(x: int) -> pd.Series:

        sr = pd.Series([0,0,0,0,0,0,0],index=['元旦','春節','二二八','兒童清明','端午','中秋','雙十'])

        def _2018( x, sr):
            if x == 20180101:
                sr['元旦'] = 1
            elif 20180215 <= x <= 20180220:
                sr['春節'] = 1
            elif x == 20180228:
                sr['二二八'] = 1
            elif 20180404 <= x <= 20180408:
                sr['兒童清明'] = 1
            elif 20180616 <= x <= 20180618:
                sr['端午'] = 1
            elif 20180922 <= x <= 20180924:
                sr['中秋'] = 1
            elif x == 20181010:
                sr['雙十'] = 1
            elif 20181229 <= x <= 20181231:
                sr['元旦'] = 1
            return sr

        def _2019( x, sr):
            if x == 20190101:
                sr['元旦'] = 1
            elif 20190202 <= x <= 20190210:
                sr['春節'] = 1
            elif 20190228 <= x <= 20190303:
                sr['二二八'] = 1
            elif 20190404 <= x <= 20190407:
                sr['兒童清明'] = 1
            elif 20190607 <= x <= 20190609:
                sr['端午'] = 1
            elif 20190913 <= x <= 20190915:
                sr['中秋'] = 1
            elif 20191010 <= x <= 20191013:
                sr['雙十'] = 1
            return sr


        def _2020( x, sr):
            if x == 20200101:
                sr['元旦'] = 1
            elif 20200123 <= x <= 20200129:
                sr['春節'] = 1
            elif 20200228 <= x <= 20200301:
                sr['二二八'] = 1
            elif 20200402 <= x <= 20200405:
                sr['兒童清明'] = 1
            elif 20200625 <= x <= 20200628:
                sr['端午'] = 1
            elif 20201001 <= x <= 20201004:
                sr['中秋'] = 1
            elif 20201009 <= x <= 20201011:
                sr['雙十'] = 1
            return sr

        def _2021( x, sr):
            if 20210101 <= x <= 20210103:
                sr['元旦'] = 1
            elif 20210210  <= x <= 20210216:
                sr['春節'] = 1
            elif 20210227 <= x <= 20210301:
                sr['二二八'] = 1
            elif 20210402 <= x <= 20210405:
                sr['兒童清明'] = 1
            elif 20210612 <= x <= 20210614:
                sr['端午'] = 1
            elif 20210918 <= x <= 20210921:
                sr['中秋'] = 1
            elif 20211009 <= x <= 20211011:
                sr['雙十'] = 1
            elif x == 20211231:
                sr['元旦'] = 1
            return sr

        def _2022( x, sr):
            if 20220101 <= x <= 20220102:
                sr['元旦'] = 1
            elif 20220129 <= x <= 20220206:
                sr['春節'] = 1
            elif 20220226 <= x <= 20220228:
                sr['二二八'] = 1
            elif 20220402 <= x <= 20220405:
                sr['兒童清明'] = 1
            elif 20220603 <= x <= 20220605:
                sr['端午'] = 1
            elif 20220909 <= x <= 20220911:
                sr['中秋'] = 1
            elif 20221008 <= x <= 20221010:
                sr['雙十'] = 1
            elif x == 20221231:
                sr['元旦'] = 1
            return sr

        def _2023( x, sr):
            if 20230101 <= x <= 20230102:
                sr['元旦'] = 1
            elif 20230120 <= x <= 20230129:
                sr['春節'] = 1
            elif 20230225 <= x <= 20230228:
                sr['二二八'] = 1
            elif 20230401 <= x <= 20230405:
                sr['兒童清明'] = 1
            elif 20230622 <= x <= 20230625:
                sr['端午'] = 1
            elif 20230929 <= x <= 20231001:
                sr['中秋'] = 1
            elif 20231007 <= x <= 20231010:
                sr['雙十'] = 1
            elif 20231230 <= x <= 20231231:
                sr['元旦'] = 1
            return sr

        year = x // 10000

        if year == 2018:
            sr = _2018(x, sr)
        elif year == 2019:
            sr = _2019(x, sr)
        elif year == 2020:
            sr = _2020(x, sr)
        elif year == 2021:
            sr = _2021(x, sr)
        elif year == 2022:
            sr = _2022(x, sr)
        elif year == 2023:
            sr = _2023(x, sr)

        return sr

    date_int = df.發生年度.astype(int)*10000 + df.發生月份.astype(int)*100 + df.發生日期.astype(int)
    df[['元旦', '春節', '二二八', '兒童清明', '端午', '中秋', '雙十']] = date_int.apply((taiwan_festival))

    def nearest(x: pd.DataFrame):
        distance = KDTree_stream.query((x.經度, x.緯度),k=3)[0]
        index = KDTree_stream.query((x.經度, x.緯度),k=3)[1]
        df = stream.iloc[index,:]
        df['與事故點距離'] = pd.Series(distance,index=df.index)
        df = df[['調查站_座標_經度','調查站_座標_緯度','與事故點距離','總計_流量(PCU)']]
        df = pd.concat([df.iloc[0,],df.iloc[1,],df.iloc[2,]])
        return df

    stream_data = df.apply(nearest, axis=1)
    stream_data.columns = ['調查站1_座標_經度','調查站1_座標_緯度','調查站1與事故點距離','調查站1總計_流量(PCU)','調查站2_座標_經度','調查站2_座標_緯度','調查站2與事故點距離','調查站2總計_流量(PCU)','調查站3_座標_經度','調查站3_座標_緯度','調查站3與事故點距離','調查站3總計_流量(PCU)']

    stream_data['流量加權'] = stream_data['調查站1總計_流量(PCU)']*(1/(stream_data.調查站1與事故點距離))/(1/(stream_data.調查站1與事故點距離)+1/(stream_data.調查站2與事故點距離)+1/(stream_data.調查站3與事故點距離)) +\
                stream_data['調查站2總計_流量(PCU)']*(1/(stream_data.調查站2與事故點距離))/(1/(stream_data.調查站1與事故點距離)+1/(stream_data.調查站2與事故點距離)+1/(stream_data.調查站3與事故點距離)) +\
                stream_data['調查站3總計_流量(PCU)']*(1/(stream_data.調查站3與事故點距離))/(1/(stream_data.調查站1與事故點距離)+1/(stream_data.調查站2與事故點距離)+1/(stream_data.調查站3與事故點距離))

    df.流量加權 = stream_data.流量加權
    
    return df

# https://www.visualcrossing.com
# def weatherAPI(df1):
#     # 密鑰
# #     key = '4KD3GK5WQ8HX23YXVEVS3ZYPM'
#     key = 'NZKHHNNZV9ZXDRGDM4NJTT9YR'
#     # 緯度，經度
#     a1 = f"{df1.iloc[0,14]},{df1.iloc[0,13]}"
#     # 時間
#     t1 = f"{df1.iloc[0,9]}-{df1.iloc[0,10]}-{df1.iloc[0,11]}"
#     t2 = f"{df1.iloc[0,9]}-{df1.iloc[0,10]}-{df1.iloc[0,11]}"
#     # 單位
#     unitGroup = 'metric'
#     # 語言
#     lang = 'zh'
#     # 需求資料
#     include = 'hours'
#     # 元素
#     elements = 'datetime,pressure,humidity,temp,winddir,windspeed,precip'

#     url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{a1}/{t1}/{t2}?key={key}&unitGroup={unitGroup}&lang={lang}&include={include}&elements={elements}"


#     response = requests.get(url)
# #     print(response.status_code)
#     if response.status_code == 200:
#         data = json.loads(response.text)
# #         pprint(data)
#         a = pd.DataFrame(data['days'][0]['hours'])
#         df1['氣溫(℃)_x'] = a['temp']
#         df1['測站氣壓(hPa)_x'] = a['pressure']
#         df1['相對溼度(%)_x'] = a['humidity']
#         df1['降水量(mm)_x'] = a['precip']
#         df1['風向(360degree)_x'] = a['winddir']
#         df1['風速(m/s)_x'] = a['windspeed']*1000/3600
#         df1 = df1.round({'風速(m/s)_x':1})
#     return df1
def weatherAPI(df1):
    # 密鑰
#     key = '4KD3GK5WQ8HX23YXVEVS3ZYPM'
    key = 'NZKHHNNZV9ZXDRGDM4NJTT9YR'
    # 緯度，經度
    a1 = f"{df1.iloc[0,14]},{df1.iloc[0,13]}"
    # 時間
    t1 = f"{df1.iloc[0,9]}-{df1.iloc[0,10]}-{df1.iloc[0,11]}"
    day = datetime.datetime.strptime(t1, '%Y-%m-%d')
    delta = datetime.timedelta(days=1)
    day1 = day + delta
    t2 = day1.strftime('%Y-%m-%d')
    # 單位
    unitGroup = 'metric'
    # 語言
    lang = 'zh'
    # 需求資料
    include = 'hours'
    # 元素
    elements = 'datetime,pressure,humidity,temp,winddir,windspeed,precip'

    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{a1}/{t1}/{t2}?key={key}&unitGroup={unitGroup}&lang={lang}&include={include}&elements={elements}"


    response = requests.get(url)
#     print(response.status_code)
    if response.status_code == 200:
        data = json.loads(response.text)
        a = pd.DataFrame(data['days'][0]['hours'])
        b = pd.DataFrame(data['days'][1]['hours'])
        c = pd.concat([a,b],axis=0)
        c.reset_index(drop=True, inplace=True)
        d = c.iloc[df1.發生時間[0]:df1.發生時間[0]+24,:]
        d.reset_index(drop=True, inplace=True)
        df1['氣溫(℃)_x'] = d['temp']
        df1['測站氣壓(hPa)_x'] = d['pressure']
        df1['相對溼度(%)_x'] = d['humidity']
        df1['降水量(mm)_x'] = d['precip']
        df1['風向(360degree)_x'] = d['winddir']
        df1['風速(m/s)_x'] = d['windspeed']*1000/3600
        df1 = df1.round({'風速(m/s)_x':1})
        return df1

C:\Users\student\AppData\Local\Temp\ipykernel_3964\3713644182.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stream = stream.groupby('ID').mean()


In [5]:
coo = geocoder.arcgis('新北市烏來區71號').latlng
xgdic = population('新北市烏來區',coo)
df1 = dataFill(xgdic)
forxg = weatherAPI(df1)

C:\Users\student\AppData\Local\Temp\ipykernel_3964\3713644182.py:55: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  time = pd.datetime.strptime(datetime[1], '%Y-%m-%d %H:%M:%S')


In [7]:
xgdic

{'year': '2023',
 'month': '1',
 'day': '5',
 'hour': '0',
 'long': '121.54746000000011',
 'lat': '24.87303000000003',
 'pop': '19.66'}

In [6]:
forxg.發生時間

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
Name: 發生時間, dtype: int64

In [55]:
forxg

,dow,doy,元旦,春節,二二八,兒童清明,端午,中秋,雙十,發生年度,...,經度,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權
0,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,17.2,1021.0,60.53,0.0,284.4,2.7,12.61,3914.543789
1,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,15.8,1021.0,70.28,0.0,288.8,2.1,12.61,3914.543789
2,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,13.4,1022.0,86.54,0.0,292.9,1.2,12.61,3914.543789
3,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,11.7,1023.0,91.12,0.0,104.6,1.1,12.61,3914.543789
4,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,11.6,1023.0,88.12,0.0,120.0,1.3,12.61,3914.543789
5,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,10.9,1024.0,88.65,0.0,109.3,1.8,12.61,3914.543789
6,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,10.8,1024.0,86.28,0.0,106.0,2.0,12.61,3914.543789
7,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,10.8,1025.0,83.97,0.0,109.4,2.0,12.61,3914.543789
8,3,5,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,10.3,1025.0,84.49,0.0,108.1,2.2,12.61,3914.543789
9,4,6,0,0,0,0,0,0,0,2023,...,120.7181700000001,23.46705000000003,10.1,1024.0,82.75,0.0,108.1,2.4,12.61,3914.543789


In [16]:
import pytz
from datetime import datetime,timedelta


tz = pytz.timezone('Asia/Taipei')
    # 取得當前日期
today = datetime.now(tz)

print(today.year)

print(today)

# today2 = datetime.datetime.today()
# print(today2)


delta = datetime.now(tz)+ timedelta(days=1)
print(delta)

2023
2023-01-10 15:05:04.879416+08:00
2023-01-11 15:05:04.879416+08:00
